In [28]:
import sys
sys.path.insert(0,'/home/dan/Projects/kaggle-ieee-cis-fraud-detection/src/')
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from cross_val_labo import CrossValLabo

In [20]:
train_set = pd.read_pickle('./train_set.pkl')
train_label = pd.read_pickle('./train_label.pkl')

In [21]:
val_mask = train_set.TransactionDT >= 13000000
train_mask = train_set.TransactionDT < 13000000

X_val = train_set[val_mask]
y_val = train_label[val_mask]
X_train = train_set[train_mask]
y_train = train_label[train_mask]

del train_set
del train_label

In [24]:
lgb_params = {
                'objective':'regression',
                'boosting_type':'gbdt',
                'metric':'auc',
                'n_jobs':8,
                'learning_rate':0.01,
                'num_leaves': 2**10,
                'min_child_samples': 40,
                'max_depth':12,
                'tree_learner':'serial',
                'colsample_bytree': 0.80,
                'n_estimators':2**10,
                'max_bin':255,
                'seed': 1337,
                'reg_alpha':0.3,
                'reg_lamdba':0.243
            }
model = lgb.LGBMClassifier(**lgb_params)

In [25]:
model.fit(X_train, y_train, eval_set=[])

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.8,
               importance_type='split', learning_rate=0.01, max_bin=255,
               max_depth=12, metric='auc', min_child_samples=40,
               min_child_weight=0.001, min_split_gain=0.0, n_estimators=1024,
               n_jobs=8, num_leaves=1024, objective='regression',
               random_state=None, reg_alpha=0.3, reg_lambda=0.0,
               reg_lamdba=0.243, seed=1337, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0,
               tree_learner='serial')

In [38]:
preds = model.predict_proba(X_val)[:, 1]

In [40]:
score = roc_auc_score(y_val, preds)
score

0.896726283454384